In [1]:
import itertools
import pandas as pd
import cobra.flux_analysis
from cobra.flux_analysis import flux_variability_analysis, find_essential_genes
from cobra import Metabolite
import time
import numpy as np
from src.mp_functions import combinations_subset, parallelize_dataframe, knockout_FBA, knockout_FBA_w_tasks, knockout_FBA_simple

from functools import partial
from src.task_functions import constrain_model, read_tasks


In [2]:
tissue = 'brain'
# Single model
model_file_path = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/GTEx/{0}.xml'.format(tissue)

In [3]:
#Simple Load
model = cobra.io.read_sbml_model(model_file_path)

Academic license - for non-commercial use only - expires 2022-10-03
Using license file c:\gurobi\gurobi.lic


In [3]:
model_list = constrain_model(model_file_path, ALLMETSIN_OUT=True)


##################

Academic license - for non-commercial use only - expires 2022-10-03
Using license file c:\gurobi\gurobi.lic
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmprsu058tv.lp
Reading time = 0.08 seconds
: 5482 rows, 15138 columns, 66608 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpl52snbau.lp
Reading time = 0.06 seconds
: 5482 rows, 15138 columns, 66608 nonzeros


In [2]:
# HIBCH FVA Test
# general model is used in this case

model_file_path = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/Human-GEM.mat'
#model = cobra.io.read_sbml_model(model_file_path)
model = cobra.io.load_matlab_model(model_file_path)

Academic license - for non-commercial use only - expires 2022-10-03
Using license file c:\gurobi\gurobi.lic


In [5]:

from cobra.flux_analysis.loopless import add_loopless, loopless_solution

#model = model_list[0]

gene = 'ENSG00000198130'

with model:


    rx_list = [model.reactions.get_by_id('MAR09046'), model.reactions.get_by_id('MAR03744')]

    #model.objective = model.reactions.get_by_id('MAR03744')

    #model.reactions.MAR03747.lower_bound = 10
    #model.reactions.MAR03747.upper_bound = 10

    # Force Valine uptake
    model.reactions.MAR09046.lower_bound = -1000
    model.reactions.MAR09046.upper_bound = -10

    # Limit intermediate exports
    model.reactions.MAR09011.upper_bound = 10
    model.reactions.MAR11481.upper_bound = 10
    model.reactions.MAR11818.upper_bound = 10
    model.reactions.MAR10185.upper_bound = 10

    #model.reactions.MAR09058.lower_bound = 1000

    # Oxygen
    model.reactions.MAR09048.lower_bound = -10
    # Glucose
    model.reactions.MAR09034.lower_bound = -6


    #model.objective = model.reactions.MAR03744

    base = model.optimize()
    print(flux_variability_analysis(model, rx_list, loopless=True))

    print(base)
    print(model.metabolites.get_by_id('MAM03135c').summary())
    print(model.summary())
    print('------------------------------------------------------------')
    t_model = model
    with t_model:
        #kn = knockout_FBA(t_model, [gene])
        # t_model.reactions.get_by_id('HMR_3747')
        t_model.genes.get_by_id(gene).knock_out()
        t_rx_list = [t_model.reactions.get_by_id('MAR09046'), t_model.reactions.get_by_id('MAR03744')]
        print(flux_variability_analysis(t_model, t_rx_list, loopless=True))
        kn = t_model.optimize()

        print(kn)
        print(t_model.metabolites.get_by_id('MAM03135c').summary())
        print(t_model.summary())

# HIBCH deficient model is forced to export isobutyrylglycine or earlier metabolite.
# The non-deficient can break it valine down.

##################

          minimum  maximum
MAR09046  -1000.0    -10.0
MAR03744      0.0   1000.0
<Solution 187.354 at 0x1d2f5c1cfd0>
MAM03135c
Formula: C5H11NO2

Producing Reactions
-------------------
Percent Flux Reaction              Definition
100.00% 67.5 MAR05089 MAM03135c <=> MAM03135s

Consuming Reactions
-------------------
Percent    Flux Reaction                                                                      Definition
 98.82%   -66.7 MAR05150         MAM01371c + MAM03082c + MAM03135c --> MAM01334c + MAM02423c + MAM02759c
  1.18% -0.7972 MAR10064 0.0043 MAM00180c + 0.0014 MAM00569c + 0.0022 MAM00913c + 0.0025 MAM00916c + ...
Objective
1.0 MAR13082 = 187.35362997658078

Uptake
------
Metabolite Reaction      Flux  C-Number C-Flux
 MAM01291s MAR00566   0.03865        22  0.00%
 MAM01771s MAR00567   0.01104        20  0.00%
 MAM01371s MAR00569      1000        10 10.54%
 MAM01741s MAR00571    0.1629        22  0.00%
 MAM01689s MAR00573    0.7674        22  0.02%
 MAM00094s MAR00575    0.

In [73]:
# Specific task test

file_path_t ='C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/valine_degradation_task.tsv'

tissue = 'pancreas'
tissues = ['spleen', 'adipose_tissue', 'adrenal_gland', 'pituitary', 'thyroid', 'blood', 'brain', 'heart', 'kidney', 'liver', 'muscle', 'nerve', 'lung', 'skin']

# , ['HIBCH', ['ENSG00000198130'], 0, []],['BCAT2', ['ENSG00000105552'], 0, []]

model_file_path = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/GTEx/{0}.xml'.format(tissue)
model_list = constrain_model(model_file_path, ALLMETSIN_IN=True)
tasks_df_t = read_tasks(file_path_t, model_list)

res = pd.DataFrame([[tissue, [], 0, []]], columns=['sample_ids', 'gene_ids', 'solution', 'tasks_results'])
res = knockout_FBA_w_tasks(tasks_df_t, model_list, res)
print(res)

Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpwrvfm1g3.lp
Reading time = 0.08 seconds
: 5597 rows, 15340 columns, 67174 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpwa4vudii.lp
Reading time = 0.07 seconds
: 5597 rows, 15340 columns, 67174 nonzeros
  sample_ids gene_ids  solution tasks_results
0   pancreas       []     75.01           [0]


In [74]:
print(tasks_df_t)

[[['ALLMETSIN', <Reaction ess_1_in_1 at 0x2175bd7ce80>, <Reaction ess_1_in_2 at 0x2175bd7cd00>], [<Reaction ess_1_out_0 at 0x2175bd7c820>, <Reaction ess_1_out_1 at 0x2175bd7cfa0>, <Reaction ess_1_out_2 at 0x2175bd7cfd0>], 'nan', 2]]


In [120]:
#######################

tissue = 'liver'
model_file_path = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/Human-GEM.xml'
model_list = constrain_model(model_file_path, ALLMETSIN_IN=False)

Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpfkhea9dp.lp
Reading time = 0.11 seconds
: 8370 rows, 26156 columns, 111730 nonzeros


In [36]:
# For testing specific tasks

file_path_t ='C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/valine_degradation_task.tsv'
task_list = read_tasks(file_path_t, model_list)
#print(task_list)
t_model = model_list[1]
with t_model:
    #task_list = read_tasks('C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/essential_tasks.tsv', model_list)
    # Constrain model
    task = task_list[24]
    for subset in [task[0], task[1]]:
        for rx in subset:
            if rx == 'ALLMETSIN':
                # Adds boundary metabolites for other reactions when ALLMETSIN is used
                for r in subset[1:]:
                    for m2 in r.metabolites:
                        for r2 in m2.reactions:
                            if r2.boundary and r2.id != r.id:
                                # Could also just remove the reactions, or set them 0, 0
                                r2.add_metabolites({Metabolite(
                                                    m2.id[:-4] + 'x[x]',
                                                    formula=m2.formula,
                                                    name=' '.join(m2.name.split(' ')[:-1]) + ' [Boundary]',
                                                    compartment='x'): 1})
                continue

            t_model.add_reaction(rx)
        if task[2] != 'nan':
            t_model.add_reaction(task[2])

    #t_model.reactions.get_by_id('ess_1_in_1').knock_out()
    #t_model.reactions.get_by_id('ess_1_in_0').lower_bound = 0
    t_model.objective = t_model.reactions.get_by_id('ess_25_out_0')
    kn = t_model.optimize()
    print(kn)
    print(t_model.summary())
    #print(t_model.metabolites.get_by_id('m01371c[c]').summary())
    #print(t_model.metabolites.get_by_id('m01285c[c]').summary())
    #print(t_model.reactions.get_by_id('ess_1_in_0'))
    #print(t_model.reactions.get_by_id('ess_1_in_0').bounds)
    #print(t_model.metabolites.get_by_id('m02674s[s]').summary())
    #print(flux_variability_analysis(t_model, ['ess_1'], loopless=True))



<Solution 10.909 at 0x175ec024f40>
Objective
1.0 ess_25_out_0 = 10.90909090909093

Uptake
------
Metabolite    Reaction  Flux  C-Number  C-Flux
m02630s[s] ess_25_in_0 836.4         0   0.00%
m01965s[s] ess_25_in_1 208.5         6 100.00%

Secretion
---------
Metabolite     Reaction   Flux  C-Number C-Flux
m01450c[c] ess_25_out_0 -10.91        27 23.55%
m01596s[s] ess_25_out_1 -956.4         1 76.45%
m02040s[s] ess_25_out_2  -1000         0  0.00%



In [ ]:
# Change constraints and/or objective after task has been applied

model = model_list[1]
with model:

    # Oxygen
    model.reactions.get_by_id('MAR09048').upper_bound = 1000
    model.reactions.get_by_id('MAR09048').lower_bound = -1000
    # Glucose
    model.reactions.get_by_id('MAR09034').upper_bound = 10
    model.reactions.get_by_id('MAR09034').lower_bound = -10
    # CO2
    model.reactions.get_by_id('MAR09058').upper_bound = 1000
    model.reactions.get_by_id('MAR09058').lower_bound = -1000
    # Water
    model.reactions.get_by_id('MAR09047').upper_bound = 1000
    model.reactions.get_by_id('MAR09047').lower_bound = -1000
    # Valine
    model.reactions.get_by_id('MAR09046').upper_bound = 1000
    model.reactions.get_by_id('MAR09046').lower_bound = 0
    # PI
    model.reactions.get_by_id('MAR09071').upper_bound = 1000
    model.reactions.get_by_id('MAR09071').lower_bound = -1000
    # H+
    model.reactions.get_by_id('MAR09079').upper_bound = 1000
    model.reactions.get_by_id('MAR09079').lower_bound = -1000
    # ADP
    model.reactions.get_by_id('MAR09255').upper_bound = 1000
    model.reactions.get_by_id('MAR09255').lower_bound = -1000
    # ATP
    model.reactions.get_by_id('MAR00569').upper_bound = 1000
    model.reactions.get_by_id('MAR00569').lower_bound = -1000
    # NH4+
    model.reactions.get_by_id('MAR11420').upper_bound = 1000
    model.reactions.get_by_id('MAR11420').lower_bound = -1000
    # H+
    model.reactions.get_by_id('MAR09075').upper_bound = 1000
    model.reactions.get_by_id('MAR09075').lower_bound = -1000

    model.genes.get_by_id('ENSG00000198130').knock_out()
    model.objective = model.reactions.get_by_id('MAR09046')
    kn = model.optimize()
    print(kn)
    print(model.reactions.get_by_id('MAR09046'))
    print(model.reactions.get_by_id('MAR09046').bounds)
    print(model.summary())

